### 05/06/2019 To-do list:
1. Calibration curve analog for regression (edited) 
2. Indicate how many points are below/above each parity line
3. Plot multiple cases side by side on the same graph (e.g. test set with 50 pts & 1500 pts)
4. Manipulating dropout

### 06/14/2019 To-do list:
1. If process gets stuck on SDT transform (tqdm) step:
   a. Cache

This document demonstrates the making, training, saving, loading, and usage of a sklearn-compliant CGCNN model.

In [2]:
%load_ext ipycache

/pylon5/ch5fq5p/zulissi/miniconda3/envs/cgcnn/lib/python3.6/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/pylon5/ch5fq5p/zulissi/miniconda3/envs/cgcnn/lib/python3.6/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [3]:
import os
import sys
#Comment/add these
sys.path.insert(0,'../')
sys.path.insert(0,'/home/zulissi/software/adamwr/')

import numpy as np
import cgcnn

import time

#Select which GPU to use if necessary
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


## Load the dataset as mongo docs

In [4]:
import random
import pickle

starttime = time.clock()
#Load a selection of documents
docs = pickle.load(open('/pylon5/ch5fq5p/zulissi/CO_docs.pkl','rb'))
random.seed(42)
random.shuffle(docs)
docs = [doc for doc in docs if -3<doc['energy']<1.0]
docs = docs[:6000]

endtime = time.clock()
print('This operation took', endtime - starttime, 's.')

This operation took 15.329999999999998 s.


In [8]:
docs[0]

{'mongo_id': ObjectId('5c0aab438456361c189f1809'),
 'mpid': 'mp-867317',
 'formula': 'CCu24OSi8',
 'miller': [1, 0, 0],
 'shift': 0.2505265391666667,
 'top': True,
 'coordination': 'Cu-Cu',
 'neighborcoord': ['Cu:Cu-Cu-Cu-Cu-Cu-Si', 'Cu:Cu-Cu-Cu-Cu-Cu-Si'],
 'nextnearestcoordination': 'Cu-Cu-Cu-Cu-Cu-Cu',
 'energy': -1.1073901300000042,
 'adsorbates': ['CO'],
 'adslab_calculation_date': datetime.datetime(2017, 12, 2, 23, 5, 0, 874000),
 'atoms': {'atoms': [{'symbol': 'C',
    'position': [1.1516596819149485, 1.1360081692746096, 24.247166134462965],
    'tag': 1,
    'index': 0,
    'charge': 0.0,
    'momentum': [0.0, 0.0, 0.0],
    'magmom': 0.0},
   {'symbol': 'O',
    'position': [1.722162292447821, 1.2062511749777953, 25.274423739748862],
    'tag': 1,
    'index': 1,
    'charge': 0.0,
    'momentum': [0.0, 0.0, 0.0],
    'magmom': 0.0},
   {'symbol': 'Cu',
    'position': [1.520768192351617, 1.05044878, 17.01380937317264],
    'tag': 0,
    'index': 2,
    'charge': 0.0,
    'mom

## Get the size of the features from the data transformer, to be used in setting up the net model

In [16]:
# %%cache SDT_list.pkl SDT_list
from torch.utils.data import Dataset, DataLoader
import mongo
from cgcnn.data import StructureData, ListDataset, StructureDataTransformer
import numpy as np
import tqdm
from sklearn.preprocessing import StandardScaler

SDT = StructureDataTransformer(atom_init_loc='../atom_init.json',
                              max_num_nbr=12,
                              step=0.2,
                              radius=1,
                              use_tag=False,
                              use_fixed_info=False,
                              use_distance=True)

SDT_out = SDT.transform(docs)

structures = SDT_out[0]

# Settings necessary to build the model (since they are size of vectors as inputs)
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]


In [21]:
SDT_out[4]

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 1.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 1.]]),
 tensor([[[1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
           1.0000e+00],
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
           1.3888e-11],
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
           1.3888e-11],
          ...,
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
           1.3888e-11],
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
           1.3888e-11],
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
           1.3888e-11]],
 
         [[1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
           1.0000e+00],
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-

In [ ]:
import multiprocess as mp
from sklearn.model_selection import ShuffleSplit

SDT_out = SDT.transform(docs)

with mp.Pool(4) as pool:
    SDT_list = list(tqdm.tqdm(pool.imap(lambda x: SDT_out[x],range(len(SDT_out)),chunksize=40),total=len(SDT_out)))

In [5]:
starttime = time.clock()
with open('distance_all_docs.pkl','wb') as fhandle:
    pickle.dump(SDT_list,fhandle)

endtime = time.clock()
print('This step took', endtime - starttime, 's to complete.')

This step took 1.0800000000000054 s to complete.


## CGCNN model with skorch to make it sklearn compliant

In [6]:
with open('distance_all_docs.pkl','rb') as opensdtlist:
    SDT_list = pickle.load(opensdtlist)
    
print(SDT_list[0])

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]]), tensor([[[1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
          1.0000e+00],
         [1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
          1.0000e+00],
         [1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
          1.0000e+00],
         ...,
         [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
          1.3888e-11],
         [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
          1.3888e-11],
         [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
          1.3888e-11]],

        [[1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
          1.0000e+00],
         [1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
       

In [9]:
from torch.optim import Adam, SGD
from sklearn.model_selection import ShuffleSplit
from skorch.callbacks import Checkpoint, LoadInitState #needs skorch 0.4.0, conda-forge version at 0.3.0 doesn't cut it
from cgcnn.data import collate_pool
from skorch import NeuralNetRegressor
from cgcnn.model_no_dropout import CrystalGraphConvNet
import torch
from cgcnn.data import MergeDataset
import skorch.callbacks.base

cuda = torch.cuda.is_available()
if cuda:
    device = torch.device("cuda")
else:
    device='cpu'

startime = time.clock()

#Make a checkpoint to save parameters every time there is a new best for validation lost
cp = Checkpoint(monitor='valid_loss_best',fn_prefix='valid_best_')

#Callback to load the checkpoint with the best validation loss at the end of training
class train_end_load_best_valid_loss(skorch.callbacks.base.Callback):
    def on_train_end(self, net, X, y):
        net.load_params('valid_best_params.pt')
        
load_best_valid_loss = train_end_load_best_valid_loss()

endtime = time.clock()
print('This step takes', endtime - startime, 's to complete.')

This step takes 0.0 s to complete.


\color{red}{This seems to be a time consuming step.}

## Example converting all the documents up front

In [10]:
starttime = time.clock()
#Make the target list
target_list = np.array([doc['energy'] for doc in docs]).reshape(-1,1)

endtime = time.clock()
print('This step takes', endtime - startime, 's to complete.')

This step takes 0.020000000000003126 s to complete.


## Shuffle and Split

In [11]:
from sklearn.model_selection import train_test_split

starttime = time.clock()
SDT_training, SDT_test, target_training, target_test = train_test_split(SDT_list, target_list, test_size=0.2)

endtime = time.clock()
print('This step takes', endtime - startime, 's to complete.')


This step takes 0.030000000000001137 s to complete.


In [15]:
SDT_training[0]

(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.]]),
 tensor([[[1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
           1.0000e+00],
          [1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
           1.0000e+00],
          [1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
           1.0000e+00],
          ...,
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
           1.3888e-11],
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
           1.3888e-11],
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-04, 1.1254e-07,
           1.3888e-11]],
 
         [[1.3888e-11, 1.1254e-07, 1.2341e-04, 1.8316e-02, 3.6788e-01,
           1.0000e+00],
          [1.0000e+00, 3.6788e-01, 1.8316e-02, 1.2341e-

# Fit the model

In [9]:
from skorch.dataset import CVSplit
from skorch.callbacks.lr_scheduler import WarmRestartLR, LRScheduler

train_test_splitter = ShuffleSplit(test_size=0.25) # , random_state=42)
LR_schedule = LRScheduler('MultiStepLR',milestones=[100],gamma=0.1)

net = NeuralNetRegressor(
    CrystalGraphConvNet,
    module__orig_atom_fea_len = orig_atom_fea_len,
    module__nbr_fea_len = nbr_fea_len,
    # module__dropout = 0.2,
    batch_size=214,
    module__classification=False,
    lr=0.0056,
    max_epochs=188, # 292
    module__atom_fea_len=46,
    module__h_fea_len=83,
    module__n_conv=8,
    module__n_h=4,
    optimizer=Adam,
    iterator_train__pin_memory=True,
    iterator_train__num_workers=0,
    iterator_train__collate_fn = collate_pool,
    iterator_train__shuffle=True,
    iterator_valid__pin_memory=True,
    iterator_valid__num_workers=0,
    iterator_valid__collate_fn = collate_pool,
    device=device,
    criterion=torch.nn.MSELoss,
#   criterion=torch.nn.L1Loss,
    dataset=MergeDataset,
    train_split = CVSplit(cv=train_test_splitter),
    callbacks=[cp, load_best_valid_loss, LR_schedule]
)

In [10]:
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

def plot(df_training, df_validation, df_test):
    f, ax = plt.subplots(figsize=(8,8))
    ax.scatter(df_training['actual_value'], df_training['predicted_value'], color='orange', 
               marker='o', alpha=0.5, label='train\nMAE=%0.2f, RMSE=%0.2f, R$^2$=%0.2f'\
                %(mean_absolute_error(df_training['actual_value'], df_training['predicted_value']), 
                  np.sqrt(mean_squared_error(df_training['actual_value'], df_training['predicted_value'])),
                  r2_score(df_training['actual_value'], df_training['predicted_value'])))

    ax.scatter(df_validation['actual_value'], df_validation['predicted_value'], color='blue', 
               marker='o', alpha=0.5, label='valid\nMAE=%0.2f, RMSE=%0.2f, R$^2$=%0.2f'\
                %(mean_absolute_error(df_validation['actual_value'], df_validation['predicted_value']), 
                  np.sqrt(mean_squared_error(df_validation['actual_value'], df_validation['predicted_value'])),
                  r2_score(df_validation['actual_value'], df_validation['predicted_value'])))

    ax.scatter(df_test['actual_value'], df_test['predicted_value'], color='green', 
               marker='o', alpha=0.5, label='test\nMAE=%0.2f, RMSE=%0.2f, R$^2$=%0.2f'\
                %(mean_absolute_error(df_test['actual_value'], df_test['predicted_value']), 
                  np.sqrt(mean_squared_error(df_test['actual_value'], df_test['predicted_value'])),
                  r2_score(df_test['actual_value'], df_test['predicted_value'])))

    ax.plot([min(df_training['actual_value']), max(df_training['actual_value'])], 
            [min(df_training['actual_value']), max(df_training['actual_value'])], 'k--')

    # format graph
    ax.tick_params(labelsize=14)
    ax.set_xlabel('DFT E (eV)', fontsize=14)
    ax.set_ylabel('CGCNN predicted E (eV)', fontsize=14)
    ax.set_title('Multi-element ', fontsize=14) 
    ax.legend(fontsize=12)

    plt.show()

In [11]:
def train(SDT_training, SDT_test, target_training, target_test, net):
    iters = 6
    tr_vl_len = len(SDT_training) # 4800
    """ VERY IMPORTANT: PICK TR_VL_LEN & ITERS s.t. ITERS DIVIDE TR_VL_LEN"""
    batchsize = tr_vl_len // iters
    
    splitter = KFold(iters, shuffle=False)
    arr_training = [[] for _ in range(iters)]
    arr_validation = []
    
    TrainingData = []
    ValidationData = []
    TestData = []
    
    for i, (train_indices, valid_indices) in enumerate(splitter.split(SDT_training)):
        net.initialize()
        net.fit(SDT_training, target_training)
        
        subdiv = [j for j in range(iters) if i != j]
        arr_validation.extend(net.predict(SDT_training)[valid_indices].reshape(-1))

        for k, j in enumerate(subdiv):
            train_segment = train_indices[(k*batchsize):((k+1)*batchsize)]
            arr_training[j].append(net.predict(SDT_training)[train_segment].reshape(-1))

        """
        validation_data = {'actual_value':np.array(target_training.reshape(-1))[valid_indices],
                           'predicted_value':net.predict(SDT_training)[valid_indices].reshape(-1)}
        dfvalidation = pd.DataFrame(validation_data)
        ValidationData.append(dfvalidation)
        """
        
        test_data = {'actual_value':np.array(target_test).reshape(-1),
                     'predicted_value':net.predict(SDT_test).reshape(-1)}
        dftest = pd.DataFrame(test_data)
        TestData.append(dftest)
        
        try:
            crude_training_data = {'actual_value':np.array(target_training).reshape(-1)[train_indices],
                                   'predicted_value':net.predict(SDT_training)[train_indices].reshape(-1)}
            crude_validation_data = {'actual_value':np.array(target_training).reshape(-1)[valid_indices],
                                     'predicted_value':net.predict(SDT_training)[valid_indices].reshape(-1)}

            dfcrudetraining   = pd.DataFrame(crude_training_data)
            dfcrudevalidation = pd.DataFrame(crude_validation_data)
            
            plot(dfcrudetraining, dfcrudevalidation, dftest)
        except:
            print("Error in plotting")
    
    print("arr_training:")
    print(arr_training)
    print("arr_validation:")
    print(arr_validation)
    """
    training_data = {'actual_value':np.array(target_training.reshape(-1))[train_indices],
                     'predicted_value':net.predict(SDT_training)[train_indices].reshape(-1)}
    test_data = {'actual_value':np.array(target_test).reshape(-1),
                 'predicted_value':net.predict(SDT_test).reshape(-1)}
    validation_data = {'actual_value':np.array(target_training.reshape(-1))[valid_indices],
                       'predicted_value':net.predict(SDT_training)[valid_indices].reshape(-1)}
    """
    arr_training = np.array(arr_training)
    arr_training = np.transpose(arr_training, (1, 0, 2))
    arr_training = np.reshape(arr_training, (iters-1, tr_vl_len))
    
    validation_data = {'actual_value':np.array(target_training).reshape(-1),
                       'predicted_value':arr_validation}
    dfvalidation = pd.DataFrame(validation_data)
    ValidationData.append(dfvalidation)
        
    for line in arr_training:
        training_data = {'actual_value':np.array(target_training).reshape(-1),
                         'predicted_value':line}
        dftraining = pd.DataFrame(training_data)
        TrainingData.append(dftraining)
        
    return TrainingData, ValidationData, TestData

In [12]:
from sklearn.model_selection import KFold

starttime = time.clock()

TrainingData, ValidationData, TestData = train(SDT_training,
                                               SDT_test,
                                               target_training,
                                               target_test, net)

TrainingData = pd.concat(TrainingData, axis=1)
ValidationData = pd.concat(ValidationData, axis=1)
TestData = pd.concat(TestData, axis=1)

endtime = time.clock()
print("Calculating the same points takes {} s.".format(endtime-starttime,))

Re-initializing module because the following parameters were re-set: atom_fea_len, classification, h_fea_len, n_conv, n_h, nbr_fea_len, orig_atom_fea_len.
Re-initializing optimizer because the following parameters were re-set: .
  epoch    train_loss    valid_loss    cp      dur
-------  ------------  ------------  ----  -------
      1        0.3472        0.3845     +  10.7199
      2        0.2748        0.3353     +  7.3261
      3        0.2301        0.3415        7.3324
      4        0.1932        0.2987     +  7.3621
      5        0.1910        0.1776     +  7.3711
      6        0.1645        0.2205        7.3593
      7        0.1423        0.1780        7.3413
      8        0.1444        0.1842        7.3330
      9        0.1319        0.1530     +  7.3442
     10        0.1123        0.2147        7.3322
     11        0.1102        0.1575        7.3166
     12        0.0974        0.1467     +  7.3720
     13        0.1121        0.1534        7.3694
     14        0.1

    148        0.0096        0.1263        6.8911
    149        0.0058        0.1248        6.8839
    150        0.0079        0.1250        6.8862
    151        0.0045        0.1274        6.8955
    152        0.0054        0.1239        6.8935
    153        0.0043        0.1233        6.8757
    154        0.0086        0.1229        6.8871
    155        0.0045        0.1264        6.8916
    156        0.0064        0.1260        6.8948
    157        0.0045        0.1241        6.8785
    158        0.0094        0.1243        6.8784
    159        0.0050        0.1238        6.8816
    160        0.0035        0.1228        6.8977
    161        0.0050        0.1217        6.8842
    162        0.0046        0.1241        6.8934
    163        0.0049        0.1244        6.8911
    164        0.0035        0.1268        6.8874
    165        0.0049        0.1215        6.8875
    166        0.0056        0.1245        6.8919
    167        0.0036        0.1220        6.8908


    104        0.0107        0.0963     +  6.6867
    105        0.0071        0.0969        6.6782
    106        0.0078        0.0975        6.6715
    107        0.0061        0.0961     +  6.6821
    108        0.0066        0.0963        6.6872
    109        0.0086        0.0976        6.6852
    110        0.0086        0.0977        6.6811
    111        0.0067        0.0976        6.6823
    112        0.0057        0.0973        6.6725
    113        0.0060        0.0969        6.6826
    114        0.0113        0.0965        6.6873
    115        0.0084        0.0979        6.6882
    116        0.0063        0.0970        6.6861
    117        0.0091        0.0985        6.6738
    118        0.0057        0.0968        6.6824
    119        0.0054        0.0978        6.6795
    120        0.0041        0.0970        6.6790
    121        0.0070        0.0970        6.6826
    122        0.0058        0.0968        6.6764
    123        0.0050        0.0980        6.6832


     62        0.0238        0.1339        6.8580
     63        0.0374        0.1227        6.7619
     64        0.0329        0.1341        6.8840
     65        0.0383        0.1396        6.7439
     66        0.0373        0.1259        6.8003
     67        0.0349        0.1413        6.7727
     68        0.0334        0.1243        6.7985
     69        0.0297        0.1188        6.8360
     70        0.0259        0.1309        6.7734
     71        0.0254        0.1335        6.6831
     72        0.0245        0.1345        6.6847
     73        0.0337        0.1228        6.6737
     74        0.0174        0.1187        6.6631
     75        0.0241        0.1282        6.6658
     76        0.0289        0.1199        6.6721
     77        0.0177        0.1294        6.6668
     78        0.0182        0.1366        6.6653
     79        0.0244        0.1185        6.6718
     80        0.0235        0.1260        6.6586
     81        0.0250        0.1212        6.6676


     20        0.0728        0.1174        6.6327
     21        0.0872        0.1279        6.6468
     22        0.0950        0.1031     +  6.6508
     23        0.0899        0.1211        6.6326
     24        0.0654        0.1211        6.6232
     25        0.0614        0.1002     +  6.6311
     26        0.0737        0.1594        6.6396
     27        0.0706        0.1065        6.6318
     28        0.0731        0.1057        6.6285
     29        0.0699        0.1074        6.6419
     30        0.0783        0.1009        6.6420
     31        0.0551        0.1134        6.6216
     32        0.0706        0.1523        6.6261
     33        0.0672        0.1202        6.6367
     34        0.0471        0.1186        6.6348
     35        0.0611        0.1012        6.6325
     36        0.0613        0.1518        6.6405
     37        0.0586        0.0990     +  6.6423
     38        0.0452        0.1385        6.6357
     39        0.0522        0.0968     +  6.6286


    178        0.0093        0.0918        6.6260
    179        0.0054        0.0917        6.6258
    180        0.0100        0.0916        6.6181
    181        0.0062        0.0918        6.6279
    182        0.0033        0.0933        6.6309
    183        0.0073        0.0926        6.6422
    184        0.0048        0.0936        6.6505
    185        0.0039        0.0922        6.6400
    186        0.0040        0.0937        6.6432
    187        0.0037        0.0922        6.6502
    188        0.0033        0.0929        6.6391
Error in plotting
Re-initializing module because the following parameters were re-set: atom_fea_len, classification, h_fea_len, n_conv, n_h, nbr_fea_len, orig_atom_fea_len.
Re-initializing optimizer because the following parameters were re-set: .
Re-initializing module because the following parameters were re-set: atom_fea_len, classification, h_fea_len, n_conv, n_h, nbr_fea_len, orig_atom_fea_len.
Re-initializing optimizer because the following 

    135        0.0055        0.1000        6.7313
    136        0.0044        0.0999        6.7182
    137        0.0081        0.0999        6.7144
    138        0.0050        0.0996        6.7164
    139        0.0077        0.0997        6.7206
    140        0.0062        0.1007        6.7177
    141        0.0111        0.1009        6.7120
    142        0.0117        0.1011        6.7160
    143        0.0066        0.1008        6.7174
    144        0.0050        0.1006        6.7202
    145        0.0068        0.1004        6.7144
    146        0.0054        0.1011        6.7154
    147        0.0055        0.0999        6.7185
    148        0.0035        0.1015        6.7189
    149        0.0055        0.1001        6.7303
    150        0.0046        0.1017        6.7292
    151        0.0070        0.1009        6.7195
    152        0.0061        0.1009        6.7286
    153        0.0109        0.1006        6.7102
    154        0.0053        0.0996        6.7169


     92        0.0181        0.1362        6.6901
     93        0.0122        0.1353        6.6836
     94        0.0178        0.1155        6.6712
     95        0.0161        0.1115        6.6735
     96        0.0234        0.1219        6.6749
     97        0.0196        0.1368        6.6794
     98        0.0186        0.1113        6.6738
     99        0.0125        0.1267        6.6654
    100        0.0133        0.1185        6.7116
    101        0.0118        0.1117        6.9078
    102        0.0121        0.1112        6.7419
    103        0.0086        0.1117        6.6845
    104        0.0064        0.1100        6.6853
    105        0.0086        0.1112        6.6801
    106        0.0118        0.1126        6.6780
    107        0.0094        0.1125        6.6905
    108        0.0068        0.1133        6.6661
    109        0.0082        0.1131        6.6681
    110        0.0075        0.1150        6.6724
    111        0.0069        0.1145        6.6753


NameError: name 'iters' is not defined

In [ ]:
"""
from sklearn.model_selection import train_test_split

TrainingData = []
ValidationData = []
TestData = []

iters = 7

starttime = time.clock()

for i in range(iters):
    # net()
    net.initialize()
    train_test_splitter = ShuffleSplit(test_size=0.25, random_state=42)
    train_indices, valid_indices = next(train_test_splitter.split(SDT_training))
    
    print("train_indices:", train_indices)
    print("valid_indices:", valid_indices)
    
    with open('no-dropout_log.txt', 'a') as logfile:
        logfile.write("Iter: %s" % (i,))
        logfile.write("train_indices: %s" % (train_indices,))
        logfile.write("train_indices: %s\n" % (train_indices,))

    net.fit(SDT_training, target_training)
    
    dftraining, dfvalidation, dftest = plot(SDT_training,
                                            SDT_test,
                                            target_training,
                                            target_test,
                                            train_indices,
                                            valid_indices, net)
    
    print('dftraining.type', dftraining.dtype)
    print('dftraining.type',dftraining.size)
    print(dftraining)
    
    TrainingData.append(dftraining)
    ValidationData.append(dfvalidation)
    TestData.append(dftest)
    
TrainingData = pd.concat(TrainingData, axis=1)
ValidationData = pd.concat(ValidationData, axis=1)
TestData = pd.concat(TestData, axis=1)

endtime = time.clock()
print("Calculating the same points {} times takes {} s.".format(iters, endtime-starttime))
"""

In [ ]:
"""
# The d20 suffix means a droupout of 20% is applied
TrainingData.to_pickle('TrData_7iters_d20.pkl')
ValidationData.to_pickle('VlData_7iters_d20.pkl')
TestData.to_pickle('TsData_7iters_d20.pkl')
"""

In [ ]:
"""
# The d30 suffix means a droupout of 30% is applied
TrainingData.to_pickle('TrData_7iters_d30.pkl')
ValidationData.to_pickle('VlData_7iters_d30.pkl')
TestData.to_pickle('TsData_7iters_d30.pkl')
"""

In [13]:
TrainingData.to_pickle('TrData_7iters_vanilla.pkl')
ValidationData.to_pickle('VlData_7iters_vanilla.pkl')
TestData.to_pickle('TsData_7iters_vanilla.pkl')
